In [5]:
!pip install langchain langchain-openai langgraph python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)


# Setup & Agent Creation

## BaseTool 상속 방식 (정석)

`@tool` 데코레이터 대신 `BaseTool` 클래스를 상속하는 방식의 장점:
- **명시적 구조**: 클래스 기반으로 도구의 구조가 명확함
- **타입 안정성**: Pydantic 모델로 입력 검증 가능
- **확장성**: 상속/오버라이드로 커스터마이징 용이
- **디버깅**: 클래스 구조로 디버깅이 쉬움

### 구조:
1. **입력 스키마** (`BaseModel`): Pydantic으로 입력 검증
2. **도구 클래스** (`BaseTool` 상속): `_run()` 또는 `_arun()` 구현
3. **인스턴스 생성**: 도구 객체 생성 후 에이전트에 전달

# Mode 1: updates

In [18]:
print(" [updates Mode start]")
input_data = {"messages": [{"role": "user", "content": "날씨 알려줘라 ㅇㅇ"}]}

for chunk in agent.stream(input_data, stream_mode = "updates"):
    for node_name, data in chunk.items():
        print(f"현재 단계 {node_name}")
        last_msg = data['messages'][-1]
        print(f"summary {type(last_msg).__name__}")


 [updates Mode start]
현재 단계 model
summary AIMessage


# Mode 2: messages

In [17]:
print(" [messages Mode start]")
for token, metadata in agent.stream(input_data, stream_mode = "messages"):
    if token.content:
        for block in token.content_blocks:
            if block.get('text'):
                print(block['text'], end = '|', flush = True)

 [messages Mode start]
어|떤| 도시|의| 날|씨|가| 궁|금|하|신|가|요|?| 도시| 이름|을| 알려|주|시면| 해당| 도시|의| 날|씨| 정보를| 제공|해|드|리|겠습니다|.|

# Mode 3: custom

In [20]:
print("[custom Mode start]")

for chunk in agent.stream(input_data, stream_mode = "custom"):
    print(f" tool Log : {chunk}")

[custom Mode start]


# Final: 모드 결합

In [21]:
print(" [final Mode start]")

for mode, data in agent.stream(input_data, stream_mode = "final"):
    if mode =="updates":
        node = list(data.keys())[0]
        print(f"\n[system] {node} 단계 완료")
    elif mode == "custom":
        print((f" [Log] {data}"))
    elif mode == "messages":
        token, meta = data
        if token.content:
            text = "".join([b.get('text', '') for b in token.content_blocks])
            print(text, end = "", flush = True)
    

 [final Mode start]
